# 部署ModelScope模型

[ModelScope](https://www.modelscope.cn/)是一个开源的模型社区，提供了丰富的自然语言处理、计算机视觉、多模态等领域开源模型，并提供了[ModelScope library](https://github.com/modelscope/modelscope)，支持开发者可以方便得获取模型，使用模型进行推理。

PAI支持开发者将ModelScope上的模型，简单快捷得部署为在线推理服务，本文将介绍使用PAI Python SDK完成ModelScope模型的部署。

## 安装和配置SDK

我们需要首先安装PAI Python SDK以运行本示例。

In [ ]:
!python -m pip install --upgrade alipai


SDK需要配置访问阿里云服务需要的AccessKey，以及当前使用的工作空间和OSS Bucket。在PAI SDK安装之后，通过在 **命令行终端** 中执行以下命令，按照引导配置密钥、工作空间等信息。


```shell

# 以下命令，请在 命令行终端 中执行.

python -m pai.toolkit.config

```

我们可以通过以下代码验证配置是否已生效。

In [ ]:
import pai
from pai.session import get_default_session

print(pai.__version__)

sess = get_default_session()

# 获取配置的工作空间信息
assert sess.workspace_name is not None
print(sess.workspace_name)

## 部署ModelScope模型

当前示例，我们将使用ModelScope上的["CSANMT连续语义增强机器翻译-英中-通用领域-large"](https://modelscope.cn/models/damo/nlp_csanmt_translation_en2zh/summary)模型，他支持英文到中文的翻译任务。

通过ModelScope的模型详情页，我们可以获取部署模型所需要的信息，包括**模型ID**，**模型版本**，以及**任务类型**，然后通过 `pai.modelscope.ModelScopeModel` 类，创建一个ModelScope模型对象，完成模型部署。

![](../../images/modelscope-model.png)

In [ ]:
from pai.modelscope.model import ModelScopeModel

# 配置待部署的模型信息
m = ModelScopeModel(
    command="python app.py",  # 默认的ModelScope模型推理服务启动命令
    modelscope_version="latest",  # ModelScope library的版本号，latest表示最新版本
    environment_variables={
        "MODEL_ID": "damo/nlp_csanmt_translation_en2zh",  # ModelScope的模型ID
        "TASK": "translation",  # 模型的任务类型
        "REVISION": "v1.0.1",  # 模型的版本号
    },
)

In [ ]:
from pai.common.utils import random_str
from pai.predictor import Predictor

# 部署模型，在PAI-EAS创建一个推理服务
p: Predictor = m.deploy(
    service_name="ms_model_{0}".format(random_str(8)),  # 配置推理服务名称
    instance_type="ecs.gn6i-c4g1.xlarge",  # 配置推理服务实例规格
    options={
        "metadata.rpc.keepalive": 20000,  # 配置推理服务RPC超时时间: 20s
    },
)


通过以上方式部署的模型推理服务，支持通过空字符串的预测请求，获取模型的输入输出信息。

In [ ]:
from pprint import pprint
from pai.predictor import RawResponse

# 通过一个空的预测请求，获取模型的推理输入输出的数据格式
res: RawResponse = p.raw_predict(data="")

pprint(res.json())


基于以上获得的输入数据格式信息，我们可以构建相应的预测请求，发送给到推理，获取翻译结果。

In [ ]:
res = p.predict(
    # 参考以上的获得的输入输出数据格式，配置推理请求的数据
    data={
        "input": {
            "text": "Alibaba Group's mission is to let the world have no difficult business"
        }
    }
)
pprint(res)

在测试完成之后，删除推理服务，释放机器资源。

In [ ]:
# 删除推理服务
p.delete_service()